# Validating Your Data

## Figuring out what’s in your data

In [6]:
from lxml import objectify
import pandas as pd

xml = objectify.parse(open('XMLData2.xml'))
root = xml.getroot()
df = pd.DataFrame(columns=('Number', 'String', 'Boolean'))

for i in range(0,8):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['Number', 'String', 'Boolean'], 
                   [obj[0].text, obj[1].text, 
                    obj[2].text]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
    
search = pd.DataFrame.duplicated(df)

print "DF:\n",df
print
print search[search == True]

DF:
  Number  String Boolean
0      1   First    True
1      2  Second   False
2      3   Third    True
3      4  Fourth   False
4      1   First    True
5      2  Second   False
6      3   Third    True
7      4  Fourth   False

4    True
5    True
6    True
7    True
dtype: bool


## Removing duplicates

In [7]:
from lxml import objectify
import pandas as pd

xml = objectify.parse(open('XMLData2.xml'))
root = xml.getroot()
df = pd.DataFrame(columns=('Number', 'String', 'Boolean'))

for i in range(0,8):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['Number', 'String', 'Boolean'], 
                   [obj[0].text, obj[1].text, 
                    obj[2].text]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
    
print df.drop_duplicates()

  Number  String Boolean
0      1   First    True
1      2  Second   False
2      3   Third    True
3      4  Fourth   False


## Creating a data map and data plan

In [8]:
import pandas as pd

df = pd.DataFrame({'A': [0,0,0,0,0,1,1],
                   'B': [1,2,3,5,4,2,5],
                   'C': [5,3,4,1,1,2,3]})

a_group_desc = df.groupby('A').describe()
print a_group_desc

                B         C
A                          
0 count  5.000000  5.000000
  mean   3.000000  2.800000
  std    1.581139  1.788854
  min    1.000000  1.000000
  25%    2.000000  1.000000
  50%    3.000000  3.000000
  75%    4.000000  4.000000
  max    5.000000  5.000000
1 count  2.000000  2.000000
  mean   3.500000  2.500000
  std    2.121320  0.707107
  min    2.000000  2.000000
  25%    2.750000  2.250000
  50%    3.500000  2.500000
  75%    4.250000  2.750000
  max    5.000000  3.000000


In [9]:
unstacked = a_group_desc.unstack()
print unstacked

      B                                             C                     \
  count mean       std min   25%  50%   75% max count mean       std min   
A                                                                          
0     5  3.0  1.581139   1  2.00  3.0  4.00   5     5  2.8  1.788854   1   
1     2  3.5  2.121320   2  2.75  3.5  4.25   5     2  2.5  0.707107   2   

                        
    25%  50%   75% max  
A                       
0  1.00  3.0  4.00   5  
1  2.25  2.5  2.75   3  


In [23]:
#Only see Count and Mean for each series
#loc gets specific column
#Contrast df['count']['mean']
#to df.loc[:,('one','second')] which passes a nested tuple of 
#(slice(None),('one','second')) to a single call to __getitem__. 
#This allows pandas to deal with this as a single entity.
#Furthermore this order of operations can be significantly faster, and allows one to index both axes if so desired.

print unstacked.loc[:,(slice(None),['count','mean']),]
print 

#Above is same as mentioning all the locations that need to be printed
print unstacked.loc[:,(['B','C'],['count','mean'])]

      B          C     
  count mean count mean
A                      
0     5  3.0     5  2.8
1     2  3.5     2  2.5

      B          C     
  count mean count mean
A                      
0     5  3.0     5  2.8
1     2  3.5     2  2.5


# Manipulating Categorical Variables

###Check Version of Pandas

In [24]:
print pd.version.version

0.16.2


## Creating categorical variables

In [31]:
import pandas as pd

car_colors = pd.Series(['Blue', 'Red', 'Green'], dtype='category')

car_data = pd.Series(
    pd.Categorical(['Yellow', 'Green', 'Red', 'Blue', 'Purple'],
                   categories=car_colors, ordered=False))

find_entries = pd.isnull(car_data)


car_data
print car_colors
print
#print car_data
print
print find_entries[find_entries == True]

0     Blue
1      Red
2    Green
dtype: category
Categories (3, object): [Blue, Green, Red]


0    True
4    True
dtype: bool


## Renaming levels

In [37]:
import pandas as pd

car_colors = pd.Series(['Blue', 'Red', 'Green'], 
                       dtype='category')
car_data = pd.Series(
    pd.Categorical(
        ['Blue', 'Green', 'Red', 'Blue', 'Red'],
        categories=car_colors, ordered=False))


car_colors.cat.categories = ["Purple", "Yellow", "Mauve"]
car_data.cat.categories = car_colors

print pd.isnull(car_data)
print car_colors

0    False
1    False
2    False
3    False
4    False
dtype: bool
0    Purple
1     Mauve
2    Yellow
dtype: category
Categories (3, object): [Purple, Yellow, Mauve]


## Combining levels

In [40]:
import pandas as pd

car_colors = pd.Series(['Blue', 'Red', 'Green'], 
                       dtype='category')
car_data = pd.Series(
    pd.Categorical(
        ['Blue', 'Green', 'Red', 'Green', 'Red', 'Green'],
        categories=car_colors, ordered=False))

car_data.cat.categories = ["Blue_Red", "Red", "Green"]

print car_data.ix[car_data.isin(['Red'])]

car_data.ix[car_data.isin(['Red'])] = 'Blue_Red'

print
print car_data

2    Red
4    Red
dtype: category
Categories (3, object): [Blue_Red, Red, Green]

0    Blue_Red
1       Green
2    Blue_Red
3       Green
4    Blue_Red
5       Green
dtype: category
Categories (3, object): [Blue_Red, Red, Green]


# Dealing with Dates in Your Data

## Formatting time values

In [43]:
import datetime as dt

now = dt.datetime.now()

print str(now)
print now.strftime('%a, %d %B %Y')

2015-10-05 16:03:19.496000
Mon, 05 October 2015


## Using the right time transformation

In [45]:
import datetime as dt

now = dt.datetime.now()
timevalue = now + dt.timedelta(hours=2)

print now.strftime('%H:%M:%S')
print timevalue.strftime('%H:%M:%S')
print timevalue - now

16:03:30
18:03:30
2:00:00


# Dealing with Missing Data

## Finding out missing data

In [46]:
import pandas as pd
import numpy as np

s = pd.Series([1, 2, 3, np.NaN, 5, 6, None])

print s.isnull()

print
print s[s.isnull()]

0    False
1    False
2    False
3     True
4    False
5    False
6     True
dtype: bool

3   NaN
6   NaN
dtype: float64


## Encoding missingness

In [47]:
import pandas as pd
import numpy as np

s = pd.Series([1, 2, 3, np.NaN, 5, 6, None])

print s.fillna(int(s.mean()))
print
print s.dropna()

0    1
1    2
2    3
3    3
4    5
5    6
6    3
dtype: float64

0    1
1    2
2    3
4    5
5    6
dtype: float64


## Imputing missing data

In [77]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer

s = pd.Series([1, 2, 3, np.NaN, 5, 6, None])
print s
print

imp = Imputer(missing_values='NaN', strategy='mean', axis=1)
print pd.Series(imp.fit_transform(s).tolist()[0])

imp=Imputer(missing_values='NaN',strategy='mean',axis=0)
imp.fit([1, 2, 3, 4, 5, 6, 7])

x=pd.Series(imp.transform(s).tolist()[0])
print x


X = np.array([[23.56],[53.45],['NaN'],[44.44],[77.78],['NaN'],[234.44],[11.33],[79.87]])
print X
print
print

imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
print imp.fit_transform(X)


0     1
1     2
2     3
3   NaN
4     5
5     6
6   NaN
dtype: float64

0    1.0
1    2.0
2    3.0
3    3.4
4    5.0
5    6.0
6    3.4
dtype: float64
0    1
1    2
2    3
3    4
4    5
5    6
6    7
dtype: float64
[['23.56']
 ['53.45']
 ['NaN']
 ['44.44']
 ['77.78']
 ['NaN']
 ['234.44']
 ['11.33']
 ['79.87']]


[[  23.56      ]
 [  53.45      ]
 [  74.98142857]
 [  44.44      ]
 [  77.78      ]
 [  74.98142857]
 [ 234.44      ]
 [  11.33      ]
 [  79.87      ]]


In [55]:
from sklearn.base import TransformerMixin

class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

data = [
    ['a', 1, 2],
    ['b', 1, 1],
    ['b', 2, 2],
    [np.nan, np.nan, np.nan]
]

X = pd.DataFrame(data)
xt = DataFrameImputer().fit_transform(X)

print('before...')
print(X)
print('after...')
print(xt)

before...
     0   1   2
0    a   1   2
1    b   1   1
2    b   2   2
3  NaN NaN NaN
after...
   0         1         2
0  a  1.000000  2.000000
1  b  1.000000  1.000000
2  b  2.000000  2.000000
3  b  1.333333  1.666667


# Slicing and Dicing

## Slicing rows

In [14]:
x = np.array([[[1, 2, 3],  [4, 5, 6],  [7, 8, 9],],
              [[11,12,13], [14,15,16], [17,18,19],],
              [[21,22,23], [24,25,26], [27,28,29]]])

x[1]

array([[11, 12, 13],
       [14, 15, 16],
       [17, 18, 19]])

## Slicing columns

In [15]:
x = np.array([[[1, 2, 3],  [4, 5, 6],  [7, 8, 9],],
              [[11,12,13], [14,15,16], [17,18,19],],
              [[21,22,23], [24,25,26], [27,28,29]]])

x[:,1]

array([[ 4,  5,  6],
       [14, 15, 16],
       [24, 25, 26]])

## Dicing

In [16]:
x = np.array([[[1, 2, 3],  [4, 5, 6],  [7, 8, 9],],
              [[11,12,13], [14,15,16], [17,18,19],],
              [[21,22,23], [24,25,26], [27,28,29]]])

print x[1,1]
print x[:,1,1]
print x[1,:,1]
print
print x[1:3, 1:3]

[14 15 16]
[ 5 15 25]
[12 15 18]

[[[14 15 16]
  [17 18 19]]

 [[24 25 26]
  [27 28 29]]]


# Concatenating and Transforming

## Adding new cases and variables

In [17]:
import pandas as pd

df = pd.DataFrame({'A': [2,3,1],
                   'B': [1,2,3],
                   'C': [5,3,4]})

df1 = pd.DataFrame({'A': [4],
                    'B': [4],
                    'C': [4]})

df = df.append(df1)
df = df.reset_index(drop=True)
print df

df.loc[df.last_valid_index() + 1] = [5, 5, 5]
print
print df

df2 = pd.DataFrame({'D': [1, 2, 3, 4, 5]})

df = pd.DataFrame.join(df, df2)
print
print df

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4
4  5  5  5

   A  B  C  D
0  2  1  5  1
1  3  2  3  2
2  1  3  4  3
3  4  4  4  4
4  5  5  5  5


## Removing data

In [18]:
import pandas as pd

df = pd.DataFrame({'A': [2,3,1],
                   'B': [1,2,3],
                   'C': [5,3,4]})

df = df.drop(df.index[[1]])
print df

df = df.drop('B', 1)
print
print df

   A  B  C
0  2  1  5
2  1  3  4

   A  C
0  2  5
2  1  4


## Sorting and shuffling

In [19]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'A': [2,1,2,3,3,5,4],
                   'B': [1,2,3,5,4,2,5],
                   'C': [5,3,4,1,1,2,3]})

df = df.sort_index(by=['A', 'B'], ascending=[True, True])
df = df.reset_index(drop=True)
print df

index = df.index.tolist()
np.random.shuffle(index)
df = df.ix[index]
df = df.reset_index(drop=True)
print
print df

   A  B  C
0  1  2  3
1  2  1  5
2  2  3  4
3  3  4  1
4  3  5  1
5  4  5  3
6  5  2  2

   A  B  C
0  2  1  5
1  5  2  2
2  4  5  3
3  3  5  1
4  3  4  1
5  1  2  3
6  2  3  4


# Aggregating Data at Any Level

In [20]:
import pandas as pd

df = pd.DataFrame({'Map': [0,0,0,1,1,2,2],
                   'Values': [1,2,3,5,4,2,5]})

df['S'] = df.groupby('Map')['Values'].transform(np.sum)
df['M'] = df.groupby('Map')['Values'].transform(np.mean)
df['V'] = df.groupby('Map')['Values'].transform(np.var)

print df

   Map  Values  S    M    V
0    0       1  6  2.0  1.0
1    0       2  6  2.0  1.0
2    0       3  6  2.0  1.0
3    1       5  9  4.5  0.5
4    1       4  9  4.5  0.5
5    2       2  7  3.5  4.5
6    2       5  7  3.5  4.5
